In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [14]:
with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

In [3]:
from langchain_community.llms import LlamaCpp

import tomllib

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+_CONFIGS['model_mapping'][_CONFIGS['model_name']],
    name=_CONFIGS['model_name'], 
    **_CONFIGS['llm']
)

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 21 key-value pairs and 387 tensors from /Users/fred/Documents/models/qwen1_5-7b-chat-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen1.5-7B-Chat-AWQ-fp16
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 32
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_m

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    _CONFIGS["model_name"], trust_remote_code=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import tomllib

with open("../templates/query_constructor_template.toml", "rb") as f:
    templates = tomllib.load(f)

In [6]:
from my_notion_companion.few_shot_constructor import FewShotTemplateConstructor

few_shot = FewShotTemplateConstructor(tokenizer, templates)

In [9]:
s = few_shot.invoke("新裤子乐队最出名的专辑是什么时候发售的？请简单介绍该乐队。")
s

'<|im_start|>system\nYou are a helpful AI assistant, your only role is to find the keywords from a given text. These keywords will be used for subsequent search in a search engine. DO NOT answer any questions. Only extract keywords. Please strictly follow the response pattern from conversation history.\n<|im_end|>\n<|im_start|>user\n<< text >>\n中华人民共和国成立时间是?<|im_end|>\n<|im_start|>assistant\n<< search keywords >>\n中华人民共和国成立时间<|im_end|>\n<|im_start|>user\n<< text >>\n李白和杜甫相识吗？他们是怎么认识的？请从资料库中找到答案。<|im_end|>\n<|im_start|>assistant\n<< search keywords >>\n李白和杜甫的相识<|im_end|>\n<|im_start|>user\n<< text >>\n“鹅鹅鹅，曲项向天歌”是哪位诗人所作？何时所作？请只给出诗人名字。<|im_end|>\n<|im_start|>assistant\n<< search keywords >>\n鹅鹅鹅，曲项向天歌<|im_end|>\n<|im_start|>user\n<< text >>\n新裤子乐队最出名的专辑是什么时候发售的？请简单介绍该乐队。<|im_end|>\n<|im_start|>assistant\n<< search keywords >>\n新裤子乐队<|im_end|>\n<|im_start|>user\n<< text >>\n新裤子乐队最出名的专辑是什么时候发售的？请简单介绍该乐队。<|im_end|>\n<|im_start|>assistant\n<< search keywords >>\n'

In [10]:
from langchain_core.runnables import RunnableLambda
c = RunnableLambda(few_shot.invoke) | llm

In [11]:
c.invoke("“鹅鹅鹅，曲项向天歌”是哪位诗人所作？何时所作？请只给出诗人名字。")


llama_print_timings:        load time =    3154.48 ms
llama_print_timings:      sample time =      11.32 ms /    34 runs   (    0.33 ms per token,  3003.53 tokens per second)
llama_print_timings: prompt eval time =    3153.73 ms /   303 tokens (   10.41 ms per token,    96.08 tokens per second)
llama_print_timings:        eval time =    1223.03 ms /    33 runs   (   37.06 ms per token,    26.98 tokens per second)
llama_print_timings:       total time =    4606.74 ms /   336 tokens


'鹅鹅鹅，曲项向天歌\n\n诗人名字：李白\n\n时间：唐朝，具体创作时间无法确定，因为这是李白的代表作之一。'

In [12]:
from utils import load_test_cases

test_cases = load_test_cases('../data/test_cases.txt')
test_cases[:3]

[{'q': '什么是我国第一部编年史著作？', 'a': '《左传》。', 'docs': ['附：《左传》是我国第一部编年史著作。\n']},
 {'q': '什么是我国第一部编年国别史？', 'a': '《国语》。', 'docs': ['附：《国语》是我国第一部编年国别史。\n']},
 {'q': '“寡人之于国也”下一句是什么？来自哪里？',
  'a': '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  'docs': ['梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
   '《寡人之于国也》（孟子）\n']}]

In [30]:
from my_notion_companion.query_constructor import QueryConstructor

qc = QueryConstructor(llm, _CONFIGS)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
for case in test_cases:
    r = qc.invoke(case['q'])
    print(case['q'])
    print(r)
    print("-"*30)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3154.48 ms
llama_print_timings:      sample time =       2.49 ms /     7 runs   (    0.36 ms per token,  2811.24 tokens per second)
llama_print_timings: prompt eval time =    3952.80 ms /   325 tokens (   12.16 ms per token,    82.22 tokens per second)
llama_print_timings:        eval time =     202.33 ms /     6 runs   (   33.72 ms per token,    29.65 tokens per second)
llama_print_timings:       total time =    4211.21 ms /   331 tokens
Llama.generate: prefix-match hit


什么是我国第一部编年史著作？
我国第一部编年史著作
------------------------------



llama_print_timings:        load time =    3154.48 ms
llama_print_timings:      sample time =       2.98 ms /     9 runs   (    0.33 ms per token,  3017.10 tokens per second)
llama_print_timings: prompt eval time =     255.51 ms /    25 tokens (   10.22 ms per token,    97.84 tokens per second)
llama_print_timings:        eval time =     267.79 ms /     8 runs   (   33.47 ms per token,    29.87 tokens per second)
llama_print_timings:       total time =     581.69 ms /    33 tokens
Llama.generate: prefix-match hit


什么是我国第一部编年国别史？
我国第一部编年国别史著作
------------------------------



llama_print_timings:        load time =    3154.48 ms
llama_print_timings:      sample time =      14.96 ms /    45 runs   (    0.33 ms per token,  3008.22 tokens per second)
llama_print_timings: prompt eval time =     292.25 ms /    31 tokens (    9.43 ms per token,   106.07 tokens per second)
llama_print_timings:        eval time =    1467.44 ms /    44 runs   (   33.35 ms per token,    29.98 tokens per second)
llama_print_timings:       total time =    2051.67 ms /    75 tokens
Llama.generate: prefix-match hit


...

“我相信某一天，在某个人的生命中，他头顶天空中所有云朵的形状将会片刻地与他相似。”出自哪里？
我相信某一天，在某个人的生命中，他头顶天空中所有云朵的形状将会片刻地与他相似。
------------------------------



llama_print_timings:        load time =    3154.48 ms
llama_print_timings:      sample time =      16.26 ms /    49 runs   (    0.33 ms per token,  3013.16 tokens per second)
llama_print_timings: prompt eval time =     434.26 ms /    45 tokens (    9.65 ms per token,   103.62 tokens per second)
llama_print_timings:        eval time =    1828.43 ms /    48 runs   (   38.09 ms per token,    26.25 tokens per second)
llama_print_timings:       total time =    2573.50 ms /    93 tokens
Llama.generate: prefix-match hit


有一个诗人曾经写过类似“有人把国家带进沟里，但这些人说统治是一门艺术”的话。请找到出处。
有人把国家带进沟里，但这些人说统治是一门艺术
------------------------------



llama_print_timings:        load time =    3154.48 ms
llama_print_timings:      sample time =      63.73 ms /   191 runs   (    0.33 ms per token,  2996.92 tokens per second)
llama_print_timings: prompt eval time =     430.88 ms /    37 tokens (   11.65 ms per token,    85.87 tokens per second)
llama_print_timings:        eval time =    7267.34 ms /   190 runs   (   38.25 ms per token,    26.14 tokens per second)
llama_print_timings:       total time =    8925.65 ms /   227 tokens


心学和理学的区别是什么？请从《中国哲学简史》中找到相关资料。
心学 理学 区别 《中国哲学简史》
------------------------------
